In [ ]:
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

yf.pdr_override()

y_symbol_pairs = [('^GSPC', 'USD'), ('^MXX', 'MXN'), ('^FCHI', 'EUR'), ('^NYA', 'USD'),('^JKSE', 'IDR'), ('^RUT', 'USD'), ('^KS11', 'KRW')]
y_symbol_pairs = [('^GSPC', 'USD'), ('^MXX', 'MXN'), ('^HSI', 'HKD'), ('^N225', 'JPY'),('^JKSE', 'IDR'), ('^N100', 'EUR'), ('^KLSE', 'MYR')]

#y_symbols = ['^GSPC' , '^MXX', '^FCHI','^NYA','^JKSE', '^RUT', '^KS11' ]
#y_symbols = ['AAPL', 'MSFT', 'KO', 'PEP', 'GE', 'GM', 'F' ]


start_date = dt.datetime(2000,1,1)
end_date = dt.datetime(2020,1,2)


In [ ]:
def convert_to_USD(index_data, currency):
    if currency == 'USD':
        return index_data
    # get historical conversion rate to USD
    tckr = yf.Ticker(currency+"USD=X")
    df = tckr.history(start=start_date, end=end_date)
    # fixing stupid error in the data
    if currency=="IDR":
        df.loc["2010-11-01"]=df.loc["2010-11-02"]
    currency_data = df.resample('BMS').first()
    
    # convert the data
    for item in index_data.items():
        closest_date = currency_data.index[currency_data.index.get_loc(item[0], method='nearest')]
        #print(item[0])
        #print("closest date: " + str(closest_date))
        #print("before: " + str(item[1]))
        exchange_rate = currency_data.loc[closest_date]["Open"]
        #print("exchange_rate: " + str(exchange_rate))
        index_data[item[0]] = item[1]*exchange_rate
        #print("after: " + str(index_data[item[0]]))
        #print("________________________________")
    
    return index_data

In [ ]:
data = pdr.get_data_yahoo([symbol for symbol, _ in y_symbol_pairs], start=start_date, end=end_date, interval='1mo')['Adj Close']

for symbol, currency in y_symbol_pairs:
    data[symbol] = convert_to_USD(data[symbol], currency)


In [ ]:
# 10 years sliding window
sliding_windows_data = {}

for j in range(11):
    i=12*j
    returns = (data / data.shift(1))[1+i:12*10+i]
    logReturn = np.log(returns)
    #var(aX) = a^2var(X), cov(aX, aX) = a^2 cov(X,X)
    #print(returns)
    yearly_mean = 12 * logReturn.mean()
    yearly_var = 12 * logReturn.var()
    yearly_std = np.sqrt(yearly_var)
    yearly_cov = 12 * logReturn.cov()
    
    dates = (data.index[i], data.index[12*10+i])

    sliding_windows_data[str(j+1) +'_yearly_mean'] = yearly_mean
    sliding_windows_data[str(j+1) +'_yearly_var'] = yearly_var
    sliding_windows_data[str(j+1) +'_yearly_std'] = yearly_std
    sliding_windows_data[str(j+1) +'_yearly_cov'] = yearly_cov
    sliding_windows_data[str(j+1) +'_dates'] = dates
    
    

In [ ]:
#expected yearly returns:
for j in range(11):
    print("SLIDING WINDOW ", j+1)
    print(sliding_windows_data[str(j+1) +'_yearly_mean'])
    print("____________________________________________________________")

In [ ]:
#cov_matrices:
for j in range(11):
    print("SLIDING WINDOW ", j+1)
    print(sliding_windows_data[str(j+1) +'_yearly_cov'])
    print("____________________________________________________________")

In [ ]:
from pypfopt.plotting import plot_covariance
#cov_matrices:
for j in range(11):
    plot_covariance(sliding_windows_data[str(j+1) +'_yearly_cov'])

#### Efficient frontiers

In [ ]:
returns = sliding_windows_data[str(j+1) +'_yearly_mean']
[-1 if a<0 else 1 for a in returns]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

risk_free_rate = 0.01
shorting_allowed = False

 #take a list of lists of frontier returns and variances
f_r = []
f_v = []
sharper = []
#w = [] #append weights for each portfolio
frontiers_portfolios = []
highest_sharper_portfolios = []

for j in range(11):
    returns = sliding_windows_data[str(j+1) +'_yearly_mean']
    covariances = sliding_windows_data[str(j+1) +'_yearly_cov']
    shorting = [1]*len(returns)
    if shorting_allowed:
        shorting = [-1 if a<0 else 1 for a in returns]

    frontier_mean_returns = []
    frontier_stds = []
    w = []
    for i in range(3000):
        weights = np.random.normal(0, 1, size=len(returns)) #use normal distribution
        weights = [x if x >= 0 else 0 for x in weights]  
        weights /= np.sum(weights)
        # make weights negative where shorting is necessary
        weights = weights*shorting
        if np.isnan(weights).any():
            continue
        portfolio_return = np.sum(returns * weights)
        portfolio_std = np.sqrt(np.dot(weights.T, np.dot(covariances, weights)))
        frontier_mean_returns.append(portfolio_return)
        frontier_stds.append(portfolio_std)
        w.append(weights)
    frontiers_portfolios.append(w)
    f_r.append(frontier_mean_returns)
    f_v.append(frontier_stds)

    # highest sharper portfolio
    sharper.append(np.array(frontier_mean_returns) / np.array(frontier_stds))
    index = sharper[-1].argmax(axis=0)
    highest_sharper_portfolios.append({
        "dates": sliding_windows_data[str(j+1) +'_dates'],
        "return": frontier_mean_returns[index],
        "std": frontier_stds[index],
        "weights": frontiers_portfolios[j][index]})
    #print(highest_sharper_portfolios[-1]["return"])
    #print(sum(sliding_windows_data[str(j+1) +'_yearly_mean']*highest_sharper_portfolios[-1]["weights"]))

    

    
# Create a figure and a 3x3 grid of subplots
fig, axs = plt.subplots(11, figsize=(6, 50))

# Loop through the subplots and plot something on each of them
for i in range(11):
    im = axs[i].scatter(f_v[i], f_r[i], c=sharper[i])
    im = axs[i].scatter(highest_sharper_portfolios[i]["std"], highest_sharper_portfolios[i]["return"], c="red")
    # plotting capital market line (tobin separation)
    # equivalent to 
    #axs[i].axline((0, risk_free_rate), (highest_sharper_portfolios[i]["std"], highest_sharper_portfolios[i]["return"]))
    # but prettier
    axs[i].axline(
        (min(f_v[i]), (min(f_v[i])/highest_sharper_portfolios[i]["std"])*(highest_sharper_portfolios[i]["return"]-risk_free_rate)+risk_free_rate),
        (highest_sharper_portfolios[i]["std"], highest_sharper_portfolios[i]["return"]))
    axs[i].set_xlabel('Standard Deviation (Risk)')
    axs[i].set_ylabel('Expected Return')
    plt.colorbar(im, ax=axs[i], label = 'Sharpe ratio')
    
count = 0
for ax in axs.flat:
    count +=1
    ax.set_title("SLIDING WINDOW " + str(count))

plt.subplots_adjust(wspace=0.5, hspace=0.5)
# Show the plot
plt.show()



In [ ]:
# Asset Allocation (uncomment last line to prove that code works)
# I'm calculating optimal asset allocation and turnover for each time window. Maybe we're supposed to do it for each year idk

# required rate of return
rrr = 0.05
borrowing_allowed = True

# optimal asset allocation:
# mix of max sharpe ratio portfolio and risk free rate that reaches RRR

for i, portfolio in enumerate(highest_sharper_portfolios):

    risk_free_rate_portion = (rrr - portfolio["return"]) / (risk_free_rate - portfolio["return"])

    if not borrowing_allowed:
        # make sure it's between 0 and 1
        risk_free_rate_portion = 1 if risk_free_rate_portion>1 else 0 if risk_free_rate_portion<0 else risk_free_rate_portion

    highest_sharper_portfolios[i]["risk_free_rate_portion"] = risk_free_rate_portion
    highest_sharper_portfolios[i]["weights_incl_rf"] = np.append(portfolio["weights"]*(1-risk_free_rate_portion), risk_free_rate_portion)

    if risk_free_rate_portion>=0:
        print(f"""{round(risk_free_rate_portion*100)}% of the portfolio in window {str(i+1)} should be risk free at {risk_free_rate*100}%, the remaining {round((1-risk_free_rate_portion)*100)}% have a return of {round(portfolio["return"]*100)}%""")
    else:
        print(f"""Everything of the portfolio in window {str(i+1)} should be invested with a leverage of {round((1-risk_free_rate_portion),2)} (invest 100% + {round((-risk_free_rate_portion)*100)}% that was borrowed at {risk_free_rate*100}%)""")


    # check if calculation was correct
    print(f"The total return of the portfolio is {round((risk_free_rate_portion*risk_free_rate + (1-risk_free_rate_portion)*portfolio['return'])*100, 1)}%")


In [ ]:
# turnover
# For turnover I'm considering all assets, not just risk free. Not sure if it's correct.
for i, (portfolio, previous_portfolio) in enumerate(zip(highest_sharper_portfolios[1:], highest_sharper_portfolios)):
    
    # parts of the portfolio that remain the same from the previous portfolio
    unchanged_assets = [min(weight_pair) for weight_pair in zip(portfolio["weights_incl_rf"], previous_portfolio["weights_incl_rf"])]
    turnover_rate = 1-sum(unchanged_assets)
    print(f"turnover rate from sliding window {i+1} to {i+2} is {round(turnover_rate*100)}%")


In [ ]:
weights*len(weights)

In [ ]:
# Backtest (uncomment returns line and change the 12 to 119 to prove the code works (you should get aprox. 10x rrr))

monthly_portfolio_logReturns = None

for i, portfolio in enumerate(highest_sharper_portfolios[:-1]):
    
    # print year
    print(f"Year: {portfolio['dates'][1].year}")

    weights = portfolio["weights_incl_rf"]

    # connect symbol of each asset to weight the asset has in the portfolio
    symbol_weight_pairs = zip(list(data.columns)+["risk free asset"], weights)
    # remove weights that are 0
    symbol_weight_pairs = [pair for pair in symbol_weight_pairs if pair[1]]
    # print symbol weight pairs
    print(f"Weights: {', '.join( [str(round(weight*100))+'% '+symbol for symbol, weight in symbol_weight_pairs] )}")

    # return from the year that starts immediately after the sliding window
    returns = (data / data.shift(1))[i*12+120:i*12+120+12]
    #returns = (data / data.shift(1))[1+i*12:12*10+i*12] # uncomment this
    logReturn = np.log(returns)
    # add column for risk free asset
    logReturn["rf"] = [risk_free_rate/12]*12 # change last 12 to 119
    logReturn_weighted = (logReturn*weights).sum(axis=1)
    monthly_portfolio_logReturns = pd.concat([monthly_portfolio_logReturns, logReturn_weighted])
    #print(logReturn_weighted.sum(axis=1))
    #print(logReturn_weighted)
    logReturn_total = logReturn_weighted.sum()
    # total weighted return for the year
    print(f"Return: {str(round(logReturn_total*100))}%")
    
    # this works too but is more complicated
    #covariances = logReturn.cov()
    #portfolio_std = np.sqrt(np.dot(weights.T, np.dot(covariances, weights)))

    portfolio_std = np.sqrt(logReturn_weighted.var())

    print(f"Standard deviation: " + str(round(portfolio_std,4)))
    

In [ ]:
#CAPM (probably wrong idk xD)
index_symbol = 'XWD.TO' # msci world
start_date_plus_10 = start_date.replace(year = start_date.year + 10)
data_broad_market = pdr.get_data_yahoo(index_symbol, start=start_date_plus_10, end=end_date, interval='1mo')['Adj Close']

market_returns = data_broad_market / data_broad_market.shift(1)
market_logReturn = np.log(market_returns).shift(periods=-1)[:-1]
market_logReturn.name = 'market returns'
monthly_portfolio_logReturns.name = 'portfolio returns'

both_returns = pd.concat([monthly_portfolio_logReturns, market_logReturn], axis=1)

market_yearly_mean = 12 * market_logReturn.mean()
portfolio_yearly_mean = 12 * monthly_portfolio_logReturns.mean()
portfolio_yearly_var = 12 * monthly_portfolio_logReturns.var()
portfolio_yearly_cov = 12 * both_returns.cov()


beta = portfolio_yearly_cov['market returns']['portfolio returns'] / portfolio_yearly_var
market_return = market_yearly_mean # is this right??
expected_return = risk_free_rate + beta * (market_return - risk_free_rate)
jensens_alpha = portfolio_yearly_mean - (risk_free_rate + beta * (market_return - risk_free_rate))


In [ ]:
#The results might change between runs because the points are computed at random

In [ ]:
for i in range(len(f_r)):
    print("Highest return portfolio, Sliding Window ", i +1)
    print(frontiers_portfolios[i][np.argmax(f_r[i])])
    print("__________________________________________")

In [ ]:
# Less risky portfolio
for i in range(len(f_r)):
    print("Lowest risk portfolio, Sliding Window ", i +1)
    print(frontiers_portfolios[i][np.argmin(f_v[i])])
    print("__________________________________________")

In [ ]:
#Highest sharpe ratio

for i in range(11):
    SharpeR = np.array(f_r[i]) / np.array(f_v[i])
    print("Highest Sharpe-ratio portfolio, Sliding Window ", i +1)
    print(frontiers_portfolios[i][np.argmax(SharpeR)])
    print("__________________________________________")

# Let's try to replicate the previous efficient frontiers with a library

In [ ]:
!pip install PyPortfolioOpt

In [ ]:
from pypfopt import EfficientFrontier
from pypfopt import plotting

# Create a figure and a 3x3 grid of subplots
fig, axs = plt.subplots(11, figsize=(6, 50))

# Loop through the subplots and plot something on each of them
for i in range(11):
    ef = EfficientFrontier(sliding_windows_data[str(i+1)+'_yearly_mean'], sliding_windows_data[str(i+1)+'_yearly_cov'], weight_bounds=(0, 1))
    plotting.plot_efficient_frontier(ef, ax=axs[i], show_assets=True)
    axs[i].set_xlabel('Standard Deviation (Risk)')
    axs[i].set_ylabel('Expected Return')
    
count = 0
for ax in axs.flat:
    count +=1
    ax.set_title("SLIDING WINDOW " + str(count))

plt.subplots_adjust(wspace=0.5, hspace=0.5)
# Show the plot
plt.show()

### The results are the same, we can use the library to use compute the efficient frontier with risk free rate 1

## Efficient frontier risk free rate

In [ ]:
Rf = 0.01


fig, axs = plt.subplots(11, figsize=(6, 50))

# Loop through the subplots and plot something on each of them
for i in range(11):
    ef = EfficientFrontier(sliding_windows_data[str(i+1)+'_yearly_mean'], sliding_windows_data[str(i+1)+'_yearly_cov'], weight_bounds=(0, 1))
    plotting.plot_efficient_frontier(ef, ax=axs[i], show_assets=True)
    #need to create another instance
    ef2 = EfficientFrontier(sliding_windows_data[str(i+1)+'_yearly_mean'], sliding_windows_data[str(i+1)+'_yearly_cov'], weight_bounds=(0, 1))
    Best_portfolio = ef2.max_sharpe(risk_free_rate=0) #best portfolio weights
    stats = ef2.portfolio_performance(verbose=False, risk_free_rate=0)
    Rp = stats[0]
    sigma = stats[1]
    
    xs = np.linspace(0, 2)
    sigma_c = xs * sigma # slide 22 ch05
    Rc = Rf + (Rp - Rf)/sigma * sigma_c
    
    
    opt = axs[i].scatter(sigma, Rp, marker = '*', c = 'red', s=100, label = 'optimal portfolio')#best performance portfolio
    line1, = axs[i].plot(sigma_c, Rc, "-", color="k", linewidth=0.8, label = 'lending and borrowing')
    axs[i].set_xlabel('Standard Deviation (Risk)')
    axs[i].set_ylabel('Expected Return')
    axs[i].legend()
    
count = 0
for ax in axs.flat:
    count +=1
    ax.set_title("SLIDING WINDOW " + str(count))

plt.subplots_adjust(wspace=0.5, hspace=0.5)
# Show the plot
plt.show()


### 5 real bonds data

In [ ]:
#10Y coupon = 4.125, price at 14:42, 10/01/23: 104.3281 maturity = 15 NOV 2032
#30Y coupon = 4, 105.1406, 10/01/23 at 14:40 maturity = 15 NOV 2052
#5Y coupon = 3.875, price at 10/01/23, 100.6641, 14:40 maturity = 31 DEC 2027
#3Y coupon = 4, , price 99.9766 14:40 maturity = 15 DEC 2025
#2Y coupon = 4.25, price 99.9727 at 10/01/23, 14:39 maturity 31 DEC 2024
#data imported from investing.com from 01/01/2022 to 10/01/2023 (in order to get more data even if it's outside set date)

In [ ]:
prices = [99.9727, 99.9766, 100.6641, 104.3281, 105.1406 ]
coupons = [4, 4.25, 3.875, 4.125, 4]
periods = [2,3,5,10,30] #years to maturity
YTM = []
face_value = 100

### for YTM we can compute the yield with the newton method specifying the correct formula for each bond as done below

In [ ]:
from scipy.optimize import newton
#US2Y = 4 payments in 2 years
C_t = (100 * coupons[0]/100)/2
US2Y = lambda y: (C_t/((1+y/2)**1)) +(C_t/((1+y/2)**2)) + (C_t/((1+y/2)**3)) + ((face_value + C_t)/((1+y/2)**4)) - prices[0]

In [ ]:
#US3Y = 6 payments in 3 years
C_t = (100 * coupons[1]/100)/2
US3Y = lambda y: (C_t/((1+y/2)**1)) +(C_t/((1+y/2)**2)) + (C_t/((1+y/2)**3)) +\
            +(C_t/((1+y/2)**4)) + (C_t/((1+y/2)**5)) + ((face_value + C_t)/((1+y/2)**6)) - prices[1]

In [ ]:
#US5Y = 10 payments in 5 years
C_t = (100 * coupons[2]/100)/2
US5Y = lambda y: (C_t/((1+y/2)**1)) +(C_t/((1+y/2)**2)) + (C_t/((1+y/2)**3)) +\
            +(C_t/((1+y/2)**4)) + (C_t/((1+y/2)**5)) +(C_t/((1+y/2)**6)) +\
             +(C_t/((1+y/2)**7)) +(C_t/((1+y/2)**8)) +(C_t/((1+y/2)**9))+\
            + ((face_value + C_t)/((1+y/2)**10)) - prices[2]

In [ ]:
#US10Y = 20 payments in 10 years
C_t = (100 * coupons[3]/100)/2
US10Y = lambda y: (C_t/((1+y/2)**1)) +(C_t/((1+y/2)**2)) + (C_t/((1+y/2)**3)) +\
            +(C_t/((1+y/2)**4)) + (C_t/((1+y/2)**5)) +(C_t/((1+y/2)**6)) +\
             +(C_t/((1+y/2)**7))+(C_t/((1+y/2)**8))+ (C_t/((1+y/2)**9))+\
             (C_t/((1+y/2)**10)) +(C_t/((1+y/2)**11)) + (C_t/((1+y/2)**12)) +\
            +(C_t/((1+y/2)**13)) + (C_t/((1+y/2)**14)) +(C_t/((1+y/2)**15)) +\
            + (C_t/((1+y/2)**16))+ (C_t/((1+y/2)**17)) +(C_t/((1+y/2)**18))+\
            +(C_t/((1+y/2)**19)) +\
            + ((face_value + C_t)/((1+y/2)**20)) - prices[3]

In [ ]:
#US30Y = 60 payments in 30 years
C_t = (100 * coupons[4]/100)/2
US30Y = lambda y: (C_t/((1+y/2)**1)) +(C_t/((1+y/2)**2)) + (C_t/((1+y/2)**3)) +\
            +(C_t/((1+y/2)**4)) + (C_t/((1+y/2)**5)) +(C_t/((1+y/2)**6)) +\
             +(C_t/((1+y/2)**7))+(C_t/((1+y/2)**8))+ (C_t/((1+y/2)**9))+\
             (C_t/((1+y/2)**10)) +(C_t/((1+y/2)**11)) + (C_t/((1+y/2)**12)) +\
            +(C_t/((1+y/2)**13)) + (C_t/((1+y/2)**14)) +(C_t/((1+y/2)**15)) +\
            + (C_t/((1+y/2)**16))+ (C_t/((1+y/2)**17)) +(C_t/((1+y/2)**18))+\
            +(C_t/((1+y/2)**19)) +\
            (C_t/((1+y/2)**20)) +(C_t/((1+y/2)**21)) + (C_t/((1+y/2)**22)) +\
            +(C_t/((1+y/2)**23)) + (C_t/((1+y/2)**24)) +(C_t/((1+y/2)**25)) +\
             +(C_t/((1+y/2)**26))+(C_t/((1+y/2)**27))+ (C_t/((1+y/2)**28))+\
             (C_t/((1+y/2)**29)) +(C_t/((1+y/2)**30)) + (C_t/((1+y/2)**31)) +\
            +(C_t/((1+y/2)**32)) + (C_t/((1+y/2)**33)) +(C_t/((1+y/2)**34)) +\
            + (C_t/((1+y/2)**35))+ (C_t/((1+y/2)**36)) +(C_t/((1+y/2)**37))+\
            +(C_t/((1+y/2)**38)) + (C_t/((1+y/2)**39))+\
            +(C_t/((1+y/2)**40)) +(C_t/((1+y/2)**41)) + (C_t/((1+y/2)**42)) +\
            +(C_t/((1+y/2)**43)) + (C_t/((1+y/2)**44)) +(C_t/((1+y/2)**45)) +\
             +(C_t/((1+y/2)**46))+(C_t/((1+y/2)**47))+ (C_t/((1+y/2)**48))+\
             (C_t/((1+y/2)**49)) +(C_t/((1+y/2)**50)) + (C_t/((1+y/2)**51)) +\
            +(C_t/((1+y/2)**52)) + (C_t/((1+y/2)**53)) +(C_t/((1+y/2)**54)) +\
            + (C_t/((1+y/2)**55))+ (C_t/((1+y/2)**56)) +(C_t/((1+y/2)**57))+\
            +(C_t/((1+y/2)**58)) +(C_t/((1+y/2)**59))+\
            + ((face_value + C_t)/((1+y/2)**60)) - prices[4]

In [ ]:
r = round(newton(US2Y, 0.01)*100,2)
print("YTM US2Y: " + str(r))
YTM.append(r)

In [ ]:
r = round(newton(US3Y, 0.01)*100,2)
print("YTM US3Y: " + str(r))
YTM.append(r)

In [ ]:
r = round(newton(US5Y, 0.01)*100,2)
print("YTM US5Y: " + str(r))
YTM.append(r)

In [ ]:
r = round(newton(US10Y, 0.01)*100,2)
print("YTM US10Y: " + str(r))
YTM.append(r)

In [ ]:
r = round(newton(US30Y, 0.01)*100,2)
print("YTM US30Y: " + str(r))
YTM.append(r)

### We can automitize the previous procedure by using some approximation formula (in this way we can automize the yield to maturity based on period and so on ...) that return really similar values:

In [ ]:
def YTM_func(price, face, coupon, periods):
    def bond_price_calc(YTM):
        return ((coupon*((1-(1+YTM)**-periods))/YTM)+((face)/(1+YTM)**periods))-price
    return bond_price_calc

In [ ]:
for i in range(5):
    r = round(newton(YTM_func(price=prices[i],face=100,coupon=coupons[i],periods=periods[i]), 0.01)*100, 2)
    print("YTM (APPROXIMATION) BOND " +str(i) + ": " + str(r))

In [ ]:
### tutorial youtube
###https://www.investopedia.com/terms/d/duration.asp

import numpy as np
import pandas as pd
import bond_pricing

def Duration(rate, coupon_rate, frequency, face_value, settlement_date, maturity_date, change_in_yield):
    
    try:
        settlement_date = pd.to_datetime(settlement_date, format="%d/%m/%Y")
    except:
        settlement_date = pd.to_datetime(settlement_date, format="%d-%m-%Y")
        
    try:
        maturity_date = pd.to_datetime(maturity_date, format="%d/%m/%Y")
    except:
        maturity_date = pd.to_datetime(maturity_date, format="%d-%m-%Y")
    
    data = pd.DataFrame()
    rate = rate/100
    coupon_rate = coupon_rate/100
    
    #duration
    n = pd.to_numeric(((pd.to_datetime(maturity_date) - pd.to_datetime(settlement_date))/365).days)
    total_payment = n*frequency #(years times payment freq)
    coupon_payment = coupon_rate/frequency * face_value
    payment = [coupon_payment] *(total_payment-1) + [coupon_payment + face_value] ## all payments [coup, coup, ... coup+par]
    data['period'] = pd.DataFrame(np.arange(1,total_payment + 1)) # number of payments (arange skip last value)
    data['payment'] = pd.DataFrame(payment)
    data['dcoupon'] = data['payment'] / ((1 + rate/frequency)**data['period'])
    data['pv'] = data['dcoupon'] / frequency * data['period'] / data['dcoupon'].sum()#duration formula per period
    duration = data['pv'].sum() #sum to obtain duration
    m_duration = duration / (1+ (rate/frequency))
    
    
    #convexity
    #factor = 1/(data['dcoupon'].sum() * (1+rate/frequency)**2)
    
    #as in the book:
    data['cf'] = (data['payment']*(data['period']**2 + data['period'])) / (1+rate/frequency)**data['period']
    convexity = 0.5* data['cf'].sum() / data['dcoupon'].sum()
    
    #price change = duration + conv  effcts
    price_change = (-duration * change_in_yield ) + ( convexity * (change_in_yield)**2 )
    
    bond_price = data['dcoupon'].sum()#bond_pricing.simple_bonds.bond_price(settle=settlement_date, cpn=coupon_rate, mat=maturity_date, yld=rate, freq=frequency, face=face_value)
    
    corrected_price = (bond_price* price_change) + bond_price  
    
    return duration, m_duration, convexity, price_change, bond_price, corrected_price, data
    
    
    
    
    
       

# Book example

In [ ]:
Duration(10, 0, 1,1000, "01/01/2022", "01/01/2027", 0.02)
#bond_pricing.simple_bonds.bond_duration(settle="01/01/2022", mat="01/01/2032", cpn=5/100, yld=5/100, freq=1)

In [ ]:
#US2Y

In [ ]:
import matplotlib.pyplot as plt
face_value = 100

results = Duration(YTM[0], coupons[0],2, face_value, "31-12-2022", "31-12-2024", 0)
duration = results[0]
convexity = results[2]
print("BOND (US2Y) DURATION = " + str(duration))
print("BOND (US2Y) CONVEXITY = " + str(convexity))

#print how price will change with some intresets rates changes

interest_rate_shift = [-0.015, -0.01, -0.005, 0, 0.005, 0.01, 0.015]
l= []
for i in interest_rate_shift:
    l.append(Duration(YTM[0], coupons[0],2, face_value, "31-12-2022", "31-12-2024", i)[5])
plt.plot(interest_rate_shift, l, label="convexity curve")

plt.xlabel("interest change")
plt.ylabel('price')
plt.legend()

In [ ]:
#US3Y
face_value = 100

results = Duration(YTM[0], coupons[1], 2, face_value, "15-12-2022", "15-12-2025", 0)
duration = results[0]
convexity = results[2]
print("BOND (US3Y) DURATION = " + str(duration))
print("BOND (US3Y) CONVEXITY = " + str(convexity))

#print how price will change with some intresets rates changes

interest_rate_shift = [-0.015, -0.01, -0.005, 0, 0.005, 0.01, 0.015]
l= []
for i in interest_rate_shift:
    l.append(Duration(YTM[0], coupons[1], 2, face_value,  "15-12-2022", "15-12-2025", i)[5])
plt.plot(interest_rate_shift, l, label="convexity curve")

plt.xlabel("interest change")
plt.ylabel('price')
plt.legend()

In [ ]:
#US5Y
face_value = 100

results = Duration(YTM[0], coupons[2], 2, face_value, "31-12-2022", "31-12-2027", 0)
duration = results[0]
convexity = results[2]
print("BOND (US5Y) DURATION = " + str(duration))
print("BOND (US5Y) CONVEXITY = " + str(convexity))

#print how price will change with some intresets rates changes

interest_rate_shift = [-0.015, -0.01, -0.005, 0, 0.005, 0.01, 0.015]
l= []
for i in interest_rate_shift:
    l.append(Duration(YTM[0], coupons[2], 2, face_value,  "31-12-2022", "31-12-2027", i)[5])
plt.plot(interest_rate_shift, l, label="convexity curve")

plt.xlabel("interest change")
plt.ylabel('price')
plt.legend()

In [ ]:
#US10Y
face_value = 100

results = Duration(YTM[0], coupons[3], 2, face_value, "15-11-2022", "15-11-2032", 0)
duration = results[0]
convexity = results[2]
print("BOND (US10Y) DURATION = " + str(duration))
print("BOND (US10Y) CONVEXITY = " + str(convexity))

#print how price will change with some intresets rates changes

interest_rate_shift = [-0.015, -0.01, -0.005, 0, 0.005, 0.01, 0.015]
l= []
for i in interest_rate_shift:
    l.append(Duration(YTM[0], coupons[3], 2, face_value,  "15-11-2022", "15-11-2032", i)[5])
plt.plot(interest_rate_shift, l, label="convexity curve")

plt.xlabel("interest change")
plt.ylabel('price')
plt.legend()

In [ ]:
#US30Y
face_value = 100

results = Duration(YTM[0], coupons[4], 2, face_value, "15-11-2022", "15-11-2052", 0)
duration = results[0]
convexity = results[2]
print("BOND (US30Y) DURATION = " + str(duration))
print("BOND (US30Y) CONVEXITY = " + str(convexity))

#print how price will change with some intresets rates changes

interest_rate_shift = [-0.015, -0.01, -0.005, 0, 0.005, 0.01, 0.015]
l= []
for i in interest_rate_shift:
    l.append(Duration(YTM[0], coupons[4], 2, face_value,  "15-11-2022", "15-11-2052", i)[5])
plt.plot(interest_rate_shift, l, label="convexity curve")

plt.xlabel("interest change")
plt.ylabel('price')
plt.legend()

## Bonds portfolio

In [ ]:
#conversion
from forex_python.converter import CurrencyRates

c = CurrencyRates()

USD = 100000 * c.get_rate('EUR', 'USD')

In [ ]:
USD

In [ ]:
prices     #market prices

In [ ]:
bonds_portfolio = []
for p in prices:
    num_bonds = int(USD/ p)
    bonds_portfolio.append(num_bonds)
bonds_portfolio

In [ ]:
spent = np.array(bonds_portfolio) * np.array(prices)# how much was spent for each bond
#compute weights:
weights = spent/ spent.sum()
weights

In [ ]:
#Duration of portfolio:

durations = []
durations.append(round(Duration(YTM[0], coupons[0], 2, face_value, "31-12-2022", "31-12-2024", 0)[0],2))
durations.append(round(Duration(YTM[1], coupons[1], 2, face_value,  "15-12-2022", "15-12-2025", 0)[0],2))
durations.append(round(Duration(YTM[2], coupons[2], 2, face_value,  "31-12-2022", "31-12-2027", 0)[0],2))
durations.append(round(Duration(YTM[3], coupons[3], 2, face_value,  "15-11-2022", "15-11-2032", 0)[0],2))
durations.append(round(Duration(YTM[4], coupons[4], 2, face_value,  "15-11-2022", "15-11-2052", 0)[0],2))
durations

In [ ]:
#Convexity of portfolio:

convs = []
convs.append(round(Duration(YTM[0], coupons[0], 2, face_value, "31-12-2022", "31-12-2024", 0)[2],2))
convs.append(round(Duration(YTM[1], coupons[1], 2, face_value,  "15-12-2022", "15-12-2025", 0)[2],2))
convs.append(round(Duration(YTM[2], coupons[2], 2, face_value,  "31-12-2022", "31-12-2027", 0)[2],2))
convs.append(round(Duration(YTM[3], coupons[3], 2, face_value,  "15-11-2022", "15-11-2032", 0)[2],2))
convs.append(round(Duration(YTM[4], coupons[4], 2, face_value,  "15-11-2022", "15-11-2052", 0)[2],2))
convs

In [ ]:
print("Duration of portfolio: "+ str((weights * durations).sum()))

In [ ]:
print("Convexity of portfolio: "+ str((weights * convs).sum()))

### compute market value loss as yield increases

In [ ]:
#change in price with 150 basis increase in yield:
new_prices = []
new_prices.append(round(Duration(YTM[0], coupons[0], 2, face_value, "31-12-2022", "31-12-2024", 0.015)[5],2))
new_prices.append(round(Duration(YTM[1], coupons[1], 2, face_value,  "15-12-2022", "15-12-2025", 0.015)[5],2))
new_prices.append(round(Duration(YTM[2], coupons[2], 2, face_value,  "31-12-2022", "31-12-2027", 0.015)[5],2))
new_prices.append(round(Duration(YTM[3], coupons[3], 2, face_value,  "15-11-2022", "15-11-2032", 0.015)[5],2))
new_prices.append(round(Duration(YTM[4], coupons[4], 2, face_value,  "15-11-2022", "15-11-2052", 0.015)[5],2))
new_prices # new prices after change in yield

In [ ]:
price_c = (np.array(bonds_portfolio) * np.array(new_prices)).sum()

In [ ]:
ini_price = (np.array(bonds_portfolio) * np.array(prices)).sum()

In [ ]:
loss = (price_c - ini_price) / ini_price

print("the portfolio market value had decreased by: "+ str(round(loss, 4 ) * 100) + "%") 

In [ ]:
#control:
print(ini_price + (ini_price * loss))
price_c

In [ ]:
(7.5/100 * (10 / 365)) * 20000

## Computing dirty prices

In [ ]:
def dirty_price(PV, YTM, payments_per_period, t, coupon, face_value): #t=days since last payment day
    coupon = coupon/100
    interest =  coupon/payments_per_period * t/365 *face_value
    return PV + interest

In [ ]:
#prices #market prices (present value)
#YTM #yields to maturity
def compute_dirty_prices(num_payments_period, init_date, end_date,current_date, PV, YTM, coupon, face_value):
    #num_payments_period = 2
    init_date = pd.to_datetime(init_date, format="%d/%m/%Y") #day after the market prices where taken
    end_date = pd.to_datetime(end_date, format="%d/%m/%Y")
    current_date = pd.to_datetime(current_date, format="%d/%m/%Y")
    
    days_to_m = pd.to_numeric(((pd.to_datetime(end_date) - pd.to_datetime(init_date))).days) #days until maturity
    time_passed = days_to_m - pd.to_numeric(((pd.to_datetime(end_date) - pd.to_datetime(current_date))).days)
    
    years = days_to_m/365
    days_per_period = days_to_m / (years*num_payments_period)

    periods = years*num_payments_period
    dirty_prices = []
    total_days = 0
    for period in range(int(periods)):
        loop = int(days_per_period)
        if period % num_payments_period == 1: #add 1 day in odd periods, IT'S ALWAYS 2 IN MY CASE
            loop += 1
        for days_passed in range(loop):#I need to start from the 10/01
            if total_days > time_passed:
                dirty_prices.append(dirty_price(PV, YTM, num_payments_period, days_passed, coupon, face_value))
            total_days+=1
    
    dirty_prices.append(dirty_price(PV, YTM, num_payments_period, 0, coupon, face_value))#last value to complete the plot    
    return dirty_prices, time_passed, total_days

In [ ]:
d_prices, time_passed, tot_days = compute_dirty_prices(2, "31/12/2022","31/12/2024","10/01/2023", prices[0], YTM[0], coupons[0], 100)
#plt.plot(np.array(range(len(d_prices)))+10, d_prices)
#plt.plot(np.array(range(10 , 730)), d_prices) #same

fig, ax = plt.subplots(1,1)
ticks =[x for x in range(len(d_prices)) if x%100 == 0]
ticks.append(time_passed)
ticks.remove(0)
p=ax.plot(np.array(range(len(d_prices)))+10, d_prices)
ax.set_xticks(ticks)
ax.set_xticklabels(ticks)
ax.set_title("Dirty prices Bond 1")
ax.set_xlabel('Days')
ax.set_ylabel('Price')
#ax.set_xlim([10, 730])


In [ ]:
current_date = "10/01/2023"
start_dates = [ "31/12/2022","15/12/2022", "31/12/2022", "15/11/2022","15/11/2022" ]
end_dates = [ "31/12/2024","15/12/2025", "31/12/2027", "15/11/2032","15/11/2052" ]

fig, ax = plt.subplots(5, figsize=(16, 25))

for i in range(5):
    d_prices, time_passed, tot_days = compute_dirty_prices(2, start_dates[i],end_dates[i],current_date, prices[i], YTM[i], coupons[i], face_value)

    ticks =[x for x in range(len(d_prices)) if x%100 == 0]
    ticks.append(time_passed)
    ticks.remove(0)
    p=ax[i].plot(np.array(range(len(d_prices)))+10, d_prices)
    #ax[i].set_xticks(ticks)
    #ax[i].set_xticklabels(ticks)
    ax[i].set_title("Dirty prices Bond " + str(i+1))
    ax[i].set_xlabel('Days')
    ax[i].set_ylabel('Price')
    #ax.set_xlim([10, 730])



# Apply dirty prices to portfolio

In [ ]:
#plot the market value of the portfolio against days, every time a bond is matured remove it ofc.